In [13]:
%pip install imageio imageio-ffmpeg pygame casadi stable-baselines3 tensorboard "stable-baselines3[extra]"  pyvirtualdisplay ipywidgets --quiet
%pip install "gymnasium[other]" --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import gymnasium as gym
from VehicleConfigLoader import VehicleConfigLoader
from Simulation import ArticulatedVehicle
from Simulation import Map
from Simulation import MapEntity
import random
import Visualization
from casadi import cos, sin, tan
from typing import Any, SupportsFloat, Optional
from MapConfigLoader import create_default_map
from stable_baselines3 import PPO
import stable_baselines3.common.monitor
import torch
import os
import tensorboard
from IPython.display import HTML, display
from pygame import font

In [15]:
log_dir = "logs"
monitor_dir = "monitor"

In [16]:
#parametros da simulação
max_steps = 400

#### API Gymnasium

Aqui é definido o ambiente de treinamento, extendendo a classe gym.Env.

In [17]:
class ParkingEnv(gym.Env):

    metadata = {"render_modes": ["rgb_array"], "render_fps": 24}

    def __init__(self, vehicle_name: str, map: Map, dt: float = 0.5, max_steps: int = 1000):

        self.render_mode = "rgb_array"

        loader = VehicleConfigLoader("lista_veiculos.json")
        vehicle_geometry = loader.get_geometry(vehicle_name)

        self.vehicle = ArticulatedVehicle(geometry=vehicle_geometry)
        self.map = map
        self.dt = dt
        self.max_steps = max_steps
        self.steps = 0
        self.best_distance_to_goal = float('inf')
        
        # Gymnasium spaces based on README specifications
        sensor_range_m = 150.0  # meters
        speed_limit_ms = 5.0    # m/s
        steering_limit_rad = float(np.deg2rad(28.0)) # valor alpha maximo
        jackknife_limit_rad = float(np.deg2rad(65.0))

        map_width, map_height = self.map.get_size()

        # Observation: [x, y, theta, beta, alpha, r1..r14, goal_x, goal_y, goal_theta]
        obs_low = np.array(
            [
                0.0,                         # x
                0.0,                         # y
                -np.pi,                      # theta
                -jackknife_limit_rad,        # beta
                -steering_limit_rad,         # alpha
            ]
            + [0.0] * 14                     # raycasts
            + [0.0, 0.0, -np.pi],            # goal_x, goal_y, goal_theta
            dtype=np.float32,
        )

        obs_high = np.array(
            [
                map_width,                   # x
                map_height,                  # y
                np.pi,                       # theta
                jackknife_limit_rad,         # beta
                steering_limit_rad,          # alpha
            ]
            + [sensor_range_m] * 14          # raycasts
            + [map_width, map_height, np.pi],# goal_x, goal_y, goal_theta
            dtype=np.float32,
        )

        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, dtype=np.float32)

        # Action: [v, alpha]
        act_low = np.array(
            [
                1.0,             # v (allow reverse)
                -steering_limit_rad,         # alpha
            ],
            dtype=np.float32,
        )
        act_high = np.array(
            [
                speed_limit_ms,              # v
                steering_limit_rad,          # alpha
            ],
            dtype=np.float32,
        )
        self.action_space = gym.spaces.Box(low=act_low, high=act_high, dtype=np.float32)

    def reset(self, *, seed: int | None = None, options: dict | None = None):
        super().reset(seed=seed)
        self.steps = 0
        start_pos = self.map.get_random_start_position(self.vehicle, max_attempts=100)
        if start_pos is None:
            start_x = self.map.get_size()[0] / 2
            start_y = self.map.get_size()[1] / 2
        else:
            start_x, start_y = start_pos
        start_theta = 0.0
        start_beta = 0.0
        start_alpha = 0.0
        self.best_distance_to_goal = float('inf')
        self.vehicle.update_physical_properties(start_x, start_y, 0.0, start_theta, start_beta, start_alpha)
        self.vehicle.initialize_raycasts()
        self.vehicle.update_raycasts(self.map.get_entities())

        # Build observation
        x, y = self.vehicle.get_position()
        theta = self.vehicle.get_theta()
        beta = self.vehicle.get_beta()
        alpha_current = self.vehicle.get_alpha()
        raycast_lengths = [self.vehicle.raycasts[f"r{i}"].length for i in range(1, 15)]
        goal_x, goal_y = self.map.get_parking_goal_position()
        goal_theta = self.map.get_parking_goal_theta()
        observation = np.array([x, y, theta, beta, alpha_current] + raycast_lengths + [goal_x, goal_y, goal_theta], dtype=np.float32)
        info = {}
        return observation, info

    def step(self, action) -> tuple[np.ndarray, SupportsFloat, bool, bool, dict[str, Any]]:
        velocity, alpha = action
        self.steps += 1
        reward = -0.01 # recompensa base por passo de tempo (reduzida)
        terminated = False
        truncated = False
        info = {}



        ## Reward for maintaining velocity (encourages movement)
        # Linear reward for velocity magnitude, scaled appropriately
        velocity_reward = 0.05 * abs(velocity) / 5.0  # Max reward of 0.05 at max speed (5.0 m/s)
        reward += velocity_reward

        ## Strongly punish zero/low speed
        if abs(velocity) < 0.1:
            reward -= 0.4  # Penalidade muito maior para ficar parado
        elif abs(velocity) < 0.5:
            reward -= 0.05  # Penalidade menor para velocidade muito baixa

        if self.steps >= self.max_steps:
            truncated = True
            reward = -100.0
        self._move_vehicle(velocity, alpha, self.dt)
        if self._check_vehicle_collision():
            terminated = True
            reward = -20.0
        elif self._check_vehicle_parking():
            terminated = True
            reward = 100.0
        elif self._check_trailer_jackknife():
            terminated = True
            reward = -20.0
        self.vehicle.update_raycasts(self.map.get_entities())

        ## reward for getting closer to goal
        ##calculate distance to goal
        x, y = self.vehicle.get_position()
        goal_x, goal_y = self.map.get_parking_goal_position()
        distance_to_goal = np.sqrt((x - goal_x)**2 + (y - goal_y)**2)
        if distance_to_goal < self.best_distance_to_goal:
            delta = self.best_distance_to_goal - distance_to_goal
            self.best_distance_to_goal = distance_to_goal
            reward += 1.0  # Aumentado para encorajar movimento em direção ao objetivo


        # Observação: [x, y, theta, beta, alpha, r1..r14]
        x, y = self.vehicle.get_position()
        theta = self.vehicle.get_theta()
        beta = self.vehicle.get_beta()
        alpha_current = self.vehicle.get_alpha()
        raycast_lengths = [self.vehicle.raycasts[f"r{i}"].length for i in range(1, 15)]
        goal_x, goal_y = self.map.get_parking_goal_position()
        goal_theta = self.map.get_parking_goal_theta()
        observation = np.array([x, y, theta, beta, alpha_current] + raycast_lengths + [goal_x, goal_y, goal_theta], dtype=np.float32)

        return observation, reward, terminated, truncated, info

    def render(self):
        rgb_array = Visualization.to_rgb_array(self.map, self.vehicle, img_size=(288, 288))
        return rgb_array

    def close(self):
        pass


    def _move_vehicle(self, velocity: float, alpha: float, dt: float):
        # Current state
        x, y = self.vehicle.get_position()
        theta = self.vehicle.get_theta()
        beta = self.vehicle.get_beta()

        # Geometry
        D = self.vehicle.get_distancia_eixo_dianteiro_quinta_roda() - self.vehicle.get_distancia_eixo_traseiro_quinta_roda()
        L = self.vehicle.get_distancia_eixo_traseiro_trailer_quinta_roda()
        a = self.vehicle.get_distancia_eixo_traseiro_quinta_roda()

        angular_velocity_tractor = (velocity / D) * tan(alpha)
        beta_dot = angular_velocity_tractor * (1 - (alpha * cos(beta)) / L) - (velocity * sin(beta)) / L

        # Kinematics
        x_dot = velocity * cos(theta)
        y_dot = velocity * sin(theta)
        theta_dot = (velocity / D) * tan(alpha)
        beta_dot = beta_dot = angular_velocity_tractor * (1 - (alpha * cos(beta)) / L) - (velocity * sin(beta)) / L

        # Euler step
        new_x = x + x_dot * dt
        new_y = y + y_dot * dt
        new_theta = theta + theta_dot * dt
        new_beta = beta + beta_dot * dt


        self.vehicle.update_physical_properties(new_x, new_y, velocity, new_theta, new_beta, alpha)
        self.vehicle.update_raycasts(self.map.get_entities())

    def _check_vehicle_collision(self) -> bool:
        """Verifica se o veículo colidiu com alguma parede ou passou por cima de uma vaga de estacionamento."""
        for entity in self.map.get_entities():
            if entity.type == MapEntity.ENTITY_WALL:
                if self.vehicle.check_collision(entity):
                    return True
        return False

    def _check_vehicle_parking(self) -> bool:
        """Verifica se o trailer do veículo está dentro de uma vaga de estacionamento."""
        goal = self.map.get_parking_goal()
        if goal.get_bounding_box().contains_bounding_box(self.vehicle.get_bounding_box_trailer()):
            return True
        return False

    def _check_trailer_jackknife(self) -> bool:
        """Verifica se o trailer do veículo está em jackknife."""
        return self.vehicle.get_beta() > np.deg2rad(65.0) or self.vehicle.get_beta() < np.deg2rad(-65.0)



#### Funções de treinamento e avaliação

In [18]:
def evaluate_model(model: PPO, iterations: int = 10):
    total_reward = 0.0
    for _ in range(iterations):
        total_reward += run_episode(model)
    return total_reward / iterations
    
def run_episode_and_save_video(model):
    video_recorder = Visualization.VideoRecorder("simulation.mp4", fps=10)
    map = create_default_map()
    env = ParkingEnv(vehicle_name="BUG1", map=map, dt=0.5, max_steps=max_steps)
    observation, info = env.reset()
    total_reward = 0.0

    for _ in range(max_steps):
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        total_reward += float(reward)
        video_recorder.append(env.render())
        if terminated or truncated:
            break

    video_recorder.close()
    env.close()
    return total_reward

def run_episode(model):
    map = create_default_map()
    env = ParkingEnv(vehicle_name="BUG1", map=map, dt=0.5, max_steps=max_steps)
    observation, info = env.reset()
    total_reward = 0.0

    for _ in range(max_steps):
        action, _ = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        total_reward += float(reward)
        if terminated or truncated:
            break

    env.close()
    return total_reward

def run_random_episode():
    map = create_default_map()
    env = ParkingEnv(vehicle_name="BUG1", map=map, dt=0.03, max_steps=max_steps)
    observation, info = env.reset()
    total_reward = 0.0
    frames = []
    for t in range(max_steps):
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        total_reward += float(reward)
        frames.append(env.render())
        if terminated or truncated:
            break
    Visualization.save_frames_as_mp4(frames, "simulation.mp4")
    env.close()
    return total_reward

def train_ppo(model: PPO | None = None, total_timesteps: int = 10000):
    # Create environment


    map = create_default_map()

    env = ParkingEnv(vehicle_name="BUG1", map=map, dt=0.5, max_steps=max_steps)

    # Wrap the environment with a Monitor to log training progress
    # so we don't need to manually record statistics
    env = stable_baselines3.common.monitor.Monitor(env, monitor_dir)

    # neural network hyperparameters
    # net_arch is a list of number of neurons per hidden layer, e.g. [16,20] means
    # two hidden layers with 16 and 20 neurons, respectively
    policy_kwargs = dict(activation_fn=torch.nn.ReLU,
                        net_arch=[16,8])

    # instantiates the model using the defined hyperparameters
    if model is None:
        model = PPO(
            policy="MlpPolicy",           # neural network policy architecture (MLP for vector observations)
            env=env,                      # gymnasium-compatible environment to train on
            policy_kwargs=policy_kwargs,  # custom network architecture and activation
            verbose=0,                    # logging verbosity: 0(silent),1(info),2(debug)
            tensorboard_log=log_dir,      # directory for TensorBoard logs
            learning_rate=3e-4,           # optimizer learning rate
            n_steps=max_steps,                 # rollout steps per environment update
            batch_size=400,                # minibatch size for optimization
            gamma=0.9995,                   # discount factor
            gae_lambda=0.95,              # GAE lambda for bias-variance tradeoff
            ent_coef=0.01,                 # entropy coefficient (encourages exploration)
            clip_range=0.2,               # PPO clipping parameter
            n_epochs=10,                  # number of optimization epochs per update
            device="auto"                 # use GPU if available, else CPU
        )
    else:
        model.set_env(env)

    # You can also experiment with other RL algorithms like A2C, PPO, DDPG etc.
    # Refer to  https://stable-baselines3.readthedocs.io/en/master/guide/examples.html
    # for documentation. For example, if you would like to run DDPG, just replace "DQN" above with "DDPG".

    model.learn(total_timesteps=total_timesteps, progress_bar=True)

    return model

#### Funções auxiliares para visualização

In [19]:
# @title Play Video function
from IPython.display import HTML
from base64 import b64encode
import platform

# Only import and use pyvirtualdisplay on Linux
if platform.system() != 'Windows':
    from pyvirtualdisplay import Display
else:
    Display = None

# create the directory to store the video(s)
os.makedirs("./video", exist_ok=True)

# Only start virtual display on Linux (not needed on Windows)
display = None
if platform.system() != 'Windows' and Display is not None:
    display = Display(visible=False, size=(2000, 1500))
    _ = display.start()

"""
Utility functions to enable video recording of gym environment
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""
def render_mp4(videopath: str) -> str:
  """
  Gets a string containing a b4-encoded version of the MP4 video
  at the specified path.
  """
  if not os.path.exists(videopath):
      return f'<p>Video file not found: {videopath}</p>'
  mp4 = open(videopath, 'rb').read()
  base64_encoded_mp4 = b64encode(mp4).decode()
  return f'<video width=400 controls><source src="data:video/mp4;' \
         f'base64,{base64_encoded_mp4}" type="video/mp4"></video>'

def record_and_display_video_manual(env, model, video_name, num_episodes=1):
    """
    Records a video manually using Visualization.VideoRecorder (more reliable).
    
    Args:
        env: The gymnasium environment.
        model: The trained model.
        video_name (str): The name to use for the video file.
        num_episodes (int): The number of episodes to record (default is 1).
    """
    os.makedirs("./video", exist_ok=True)
    
    video_path = f"video/{video_name}.mp4"
    video_recorder = Visualization.VideoRecorder(video_path, fps=10)
    
    total_reward = 0.0
    episode_count = 0
    
    for episode in range(num_episodes):
        observation, info = env.reset()
        episode_reward = 0.0
        
        for step in range(max_steps):
            action, _ = model.predict(observation, deterministic=True)
            observation, reward, terminated, truncated, info = env.step(action)
            episode_reward += float(reward)
            video_recorder.append(env.render())
            
            if terminated or truncated:
                break
        
        total_reward += episode_reward
        episode_count += 1
    
    video_recorder.close()
    print(f"\nTotal reward: {total_reward}")
    print(f"Video saved to: {video_path}")
    
    html = render_mp4(video_path)
    return HTML(html)

def record_and_display_video(env, model, video_name, num_episodes=1):
    """
    Records a video of the agent performing in the environment and displays it.

    Args:
        env: The gymnasium environment.
        model: The trained model.
        video_name (str): The name to use for the video file.
        num_episodes (int): The number of episodes to record (default is 1).
    """
    import glob
    
    # create the directory to store the video(s)
    os.makedirs("./video", exist_ok=True)

    # Use a virtual display for rendering (only on Linux)
    display = None
    if platform.system() != 'Windows' and Display is not None:
        display = Display(visible=False, size=(1400, 900))
        _ = display.start()

    env_name = "ParkingEnv"

    env = gym.wrappers.RecordVideo(
        env,
        video_folder="video",
        name_prefix=f"{env_name}_{video_name}",
        episode_trigger=lambda episode_id: episode_id < num_episodes
    )

    observation, _ = env.reset()
    total_reward = 0
    done = False
    episode_count = 0

    while not done:
        action, states = model.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        total_reward += reward
        if done:
            episode_count += 1
            if episode_count < num_episodes:
                observation, _ = env.reset()
                done = False

    env.close()
    # Stop the virtual display if it was started
    if display is not None:
        display.stop()

    print(f"\nTotal reward: {total_reward}")

    # Find the video file that was created
    video_pattern = f"video/{env_name}_{video_name}*.mp4"
    video_files = glob.glob(video_pattern)
    
    if not video_files:
        # Try alternative pattern
        video_pattern = f"video/*{video_name}*.mp4"
        video_files = glob.glob(video_pattern)
    
    if not video_files:
        # List all video files for debugging
        all_videos = glob.glob("video/*.mp4")
        print(f"Warning: Expected video file not found. Available video files: {all_videos}")
        return HTML("<p>Video file not found. Check the video directory.</p>")
    
    # Use the first matching video file
    video_path = video_files[0]
    print(f"Found video file: {video_path}")
    
    # show video
    html = render_mp4(video_path)
    return HTML(html)

In [20]:
import shutil
def clean_logs():
    if os.path.exists(log_dir):
        print("Cleaning logs")
        shutil.rmtree(log_dir)
    os.makedirs(log_dir, exist_ok=True)

clean_logs()


Cleaning logs


In [21]:


model = None
model_save_path = "ppo_model.zip"
training_timesteps = 100000
if os.path.exists(model_save_path):
    #load and train existing model
    model = PPO.load(model_save_path)
    model = train_ppo(model, total_timesteps=training_timesteps)
    model.save(model_save_path)
else:
    #train new model
    print("Training new model")
    model = train_ppo(model, total_timesteps=training_timesteps)
    model.save(model_save_path)
print("Training complete, total timesteps: ", model._total_timesteps)

Output()

Training complete, total timesteps:  100000


In [22]:
env = ParkingEnv(vehicle_name="BUG1", map=create_default_map(), dt=0.5, max_steps=max_steps)
record_and_display_video_manual(env, model, "ppo_model", num_episodes=1)

Saved 11 frames as MP4 to video\ppo_model.mp4

Total reward: -9.0
Video saved to: video/ppo_model.mp4


In [23]:
#load tensorboard
#%load_ext tensorboard
#%tensorboard --logdir logs --host localhost --port 8096



